In [1]:
from flask import Flask, render_template, request, jsonify
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

app = Flask(__name__)

# Load Model once when the server starts
model_name = "TusharJoshi89/title-generator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Quantize for CPU speed
model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

@app.route('/')
def index():
    # Renders your landing page
    return render_template('index.html')

@app.route('/generator')
def generator_page():
    # Renders the actual generation tool page
    return render_template('generator.html')

@app.route('/api/generate', methods=['POST'])
def generate():
    data = request.json
    abstract = data.get('abstract', '')
    
    if not abstract.strip():
        return jsonify({"error": "Abstract is empty"}), 400

    input_text = "summarize: " + abstract
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    outputs = model.generate(
        inputs, 
        max_length=40, 
        num_beams=5, 
        num_return_sequences=3, 
        early_stopping=True
    )

    titles = [tokenizer.decode(out, skip_special_tokens=True).title() for out in outputs]
    return jsonify({"titles": titles})

if __name__ == '__main__':
    # Setting use_reloader=False prevents the 'SystemExit' error in Jupyter
    app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)

Loading weights:   0%|          | 0/131 [00:00<?, ?it/s]

C:\Users\User\AppData\Local\Temp\ipykernel_8732\2277307358.py:13: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Feb/2026 09:09:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2026 09:09:25] "GET /static/images/ai-cartoon1-removebg-preview.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Feb/2026 09:09:25] "GET /static/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [06/Feb/2026 09:09:27] "GET /static/images/T.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Feb/2026 09:09:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Feb/2026 09:09:30] "GET /generator HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2026 09:09:30] "GET /static/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [06/Feb/2026 09:09:30] "GET /static/images/T.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Feb/2026 09:10:28] "POST /api/generate HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2026 09:11:46] "POST /api/generate HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2026 09:14:53] "GET /generator HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2026 09:14:53] "GET /static/bootstrap/css/boot